In [1]:
import torch

In [2]:
file_path = 'exp_result/credit/Q1/0/trained_models/parties2_topmodel0_epoch300/NoDefense.pkl'

In [3]:
# Load the state_dicts and model names
state_dicts, model_names = torch.load(file_path)


In [4]:
state_dicts[0]

OrderedDict([('layer.0.weight',
              tensor([[ 0.5821,  0.3297,  0.2780,  0.3937, -0.1493, -0.2038,  0.1406,  0.0057,
                       -0.1539,  0.0777,  0.0621,  0.0357],
                      [-0.4669, -0.1752, -0.1375, -0.1674,  0.0731,  0.4063, -0.0197, -0.0800,
                        0.0418, -0.0604, -0.0403, -0.1888]], device='cuda:0')),
             ('layer.0.bias', tensor([ 1.0031, -0.8824], device='cuda:0'))])

In [7]:
from models.regression import *


In [8]:
# Instantiate the model with the correct input and output dimensions
input_dim = 12  # This should match the shape of your weight tensor
output_dim = 2  # This should match the number of biases

model = LogisticRegressionModel(input_dim, output_dim)

In [10]:
model.layer[0].bias.data = state_dicts[0]['layer.0.bias']
model.layer[0].weight.data = state_dicts[0]['layer.0.weight']

In [12]:
globals()['LogisticRegressionModel'](input_dim, output_dim)

LogisticRegressionModel(
  (layer): Sequential(
    (0): Linear(in_features=12, out_features=2, bias=True)
  )
)